In [55]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re

In [56]:
# location of raw data files
input_path = "../datasets/raw/"

In [57]:
#read PRESCRIPTIONS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'PRESCRIPTIONS.csv',dtype = "object")

In [58]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4156450 entries, 0 to 4156449
Data columns (total 19 columns):
ROW_ID               4156450 non-null object
SUBJECT_ID           4156450 non-null object
HADM_ID              4156450 non-null object
ICUSTAY_ID           2708742 non-null object
STARTDATE            4153268 non-null object
ENDDATE              4151029 non-null object
DRUG_TYPE            4156450 non-null object
DRUG                 4156450 non-null object
DRUG_NAME_POE        2492216 non-null object
DRUG_NAME_GENERIC    2493461 non-null object
FORMULARY_DRUG_CD    4154517 non-null object
GSN                  3649286 non-null object
NDC                  4151987 non-null object
PROD_STRENGTH        4155088 non-null object
DOSE_VAL_RX          4155100 non-null object
DOSE_UNIT_RX         4155108 non-null object
FORM_VAL_DISP        4155095 non-null object
FORM_UNIT_DISP       4155041 non-null object
ROUTE                4155294 non-null object
dtypes: object(19)
memory usage:

In [59]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,00469061711,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,00056017275,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,00338055002,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,00054829725,20mg Tablet,20,mg,1,TAB,PO


In [60]:
df['GSN'] = 'GSN'+'-'+df['GSN']
df['NDC'] = 'NDC'+'-'+df['GSN']


In [61]:
df['FORMULARY_DRUG_CD'] = 'DRUG_CD'+'-'+df['FORMULARY_DRUG_CD']

In [62]:
df.head(1000)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,DRUG_CD-TACR1,GSN-021796,NDC-GSN-021796,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,DRUG_CD-WARF5,GSN-006562,NDC-GSN-006562,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,DRUG_CD-HEPAPREMIX,GSN-006522,NDC-GSN-006522,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,DRUG_CD-HEPBASE,NaN,NaN,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,DRUG_CD-FURO20,GSN-008208,NDC-GSN-008208,20mg Tablet,20,mg,1,TAB,PO
5,2214774,6,107064,NaN,2175-06-11 00:00:00,2175-06-15 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,DRUG_CD-WARF0,GSN-014198,NDC-GSN-014198,Check with MD for Dose,1,dose,1,dose,PO
6,2215525,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,DRUG_CD-HEPAPREMIX,GSN-006522,NDC-GSN-006522,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
7,2216266,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,DRUG_CD-HEPBASE,NaN,NaN,HEPARIN BASE,250,ml,250,ml,IV
8,2215526,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Heparin Sodium,NaN,NaN,DRUG_CD-HEPAPREMIX,GSN-006522,NDC-GSN-006522,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
9,2214778,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,DRUG_CD-WARF2,GSN-006561,NDC-GSN-006561,2mg Tab,2,mg,1,TAB,PO


In [63]:
# Drop DRUG_NAME_POE as both are the same data
df = df.drop(labels = ['DRUG_NAME_POE'], axis = 'columns')

In [64]:
df = df.fillna('NaN')

In [65]:
# replace white space in product stength with '-'
df['PROD_STRENGTH'] = df['PROD_STRENGTH'].map(lambda x: '-'.join(str(x).split()))


In [66]:
# Merge Dosage VALUE and Dosage UNIT 
df['DOSE_UNIT_VAL_RX'] = df['DOSE_VAL_RX']+'-'+df['DOSE_UNIT_RX']
df.loc[df['DOSE_VAL_RX']=='NaN','DOSE_UNIT_VAL_RX'] = 'NaN'

In [67]:
# Merge FORM VALUE and FORM UNIT
df['FORM_VAL_UNIT_DISP'] = df['FORM_VAL_DISP']+'-'+df['FORM_UNIT_DISP']
df.loc[df['FORM_VAL_DISP']=='NaN','FORM_VAL_UNIT_DISP'] ='NaN'

In [68]:
#Drop redundant columns
df = df.drop(labels = ['DOSE_VAL_RX','DOSE_UNIT_RX','FORM_VAL_DISP','FORM_UNIT_DISP'], axis = 'columns')

In [69]:
# Replace special charecters in free text
df['DRUG'] = df['DRUG'].map(lambda x : re.sub('[!"#$&()*+,;<=>?@[\]^_`{|}~\n]', '', x))
df['DRUG_NAME_GENERIC'] = df['DRUG_NAME_GENERIC'].map(lambda x : re.sub('[!"#$&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

In [70]:
# replace white space in drug with '-' 
df['DRUG_CONCAT'] = df['DRUG'].map(lambda x: '-'.join(str(x).split()))

In [71]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,ROUTE,DOSE_UNIT_VAL_RX,FORM_VAL_UNIT_DISP,DRUG_CONCAT
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,DRUG_CD-TACR1,GSN-021796,NDC-GSN-021796,1mg-Capsule,PO,2-mg,2-CAP,Tacrolimus
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,DRUG_CD-WARF5,GSN-006562,NDC-GSN-006562,5mg-Tablet,PO,5-mg,1-TAB,Warfarin
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,DRUG_CD-HEPAPREMIX,GSN-006522,NDC-GSN-006522,"25,000-unit-Premix-Bag",IV,"25,000-UNIT",1-BAG,Heparin-Sodium
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,DRUG_CD-HEPBASE,NaN,NaN,HEPARIN-BASE,IV,250-ml,250-ml,D5W
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,DRUG_CD-FURO20,GSN-008208,NDC-GSN-008208,20mg-Tablet,PO,20-mg,1-TAB,Furosemide


In [72]:
# Concat PRODUCt STREGNTH, DOSGAE, FORM with DRUGS
df['DRUG_PROD_STRENGTH'] = df['DRUG_CONCAT']+'-'+df['PROD_STRENGTH']
df['DRUG_DOSE_UNIT_VAL_RX'] = df['DRUG_CONCAT']+'-'+df['DOSE_UNIT_VAL_RX']
df['DRUG_FORM_VAL_UNIT_DISP'] = df['DRUG_CONCAT']+'-'+df['FORM_VAL_UNIT_DISP']

In [73]:
df = df.drop(labels=['DRUG_CONCAT','PROD_STRENGTH','DOSE_UNIT_VAL_RX','FORM_VAL_UNIT_DISP'], axis = 'columns')

In [74]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,ROUTE,DRUG_PROD_STRENGTH,DRUG_DOSE_UNIT_VAL_RX,DRUG_FORM_VAL_UNIT_DISP
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,DRUG_CD-TACR1,GSN-021796,NDC-GSN-021796,PO,Tacrolimus-1mg-Capsule,Tacrolimus-2-mg,Tacrolimus-2-CAP
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,DRUG_CD-WARF5,GSN-006562,NDC-GSN-006562,PO,Warfarin-5mg-Tablet,Warfarin-5-mg,Warfarin-1-TAB
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,DRUG_CD-HEPAPREMIX,GSN-006522,NDC-GSN-006522,IV,"Heparin-Sodium-25,000-unit-Premix-Bag","Heparin-Sodium-25,000-UNIT",Heparin-Sodium-1-BAG
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,DRUG_CD-HEPBASE,NaN,NaN,IV,D5W-HEPARIN-BASE,D5W-250-ml,D5W-250-ml
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,DRUG_CD-FURO20,GSN-008208,NDC-GSN-008208,PO,Furosemide-20mg-Tablet,Furosemide-20-mg,Furosemide-1-TAB


In [75]:
df['prescriptionevents'] = df['DRUG_TYPE']+' '+df['DRUG']+' '+df['DRUG_NAME_GENERIC']+' '+df['FORMULARY_DRUG_CD']+' '+df['GSN']+' '+df['NDC']+' '+df['ROUTE']+' '+df['DRUG_PROD_STRENGTH']+' '+df['DRUG_DOSE_UNIT_VAL_RX']+' '+df['DRUG_FORM_VAL_UNIT_DISP']

In [76]:
df = df.drop(labels=['DRUG_TYPE','DRUG','DRUG_NAME_GENERIC','FORMULARY_DRUG_CD',
                    'GSN','NDC','ROUTE','DRUG_PROD_STRENGTH','DRUG_DOSE_UNIT_VAL_RX',
                     'DRUG_FORM_VAL_UNIT_DISP'], axis = 'columns')

In [77]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,prescriptionevents
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN Tacrolimus Tacrolimus DRUG_CD-TACR1 GSN-0...
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN Warfarin Warfarin DRUG_CD-WARF5 GSN-00656...
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN Heparin Sodium NaN DRUG_CD-HEPAPREMIX GSN...
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE D5W NaN DRUG_CD-HEPBASE NaN NaN IV D5W-HE...
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN Furosemide Furosemide DRUG_CD-FURO20 GSN-...


In [78]:
df['prescriptionevents'] = df['prescriptionevents'].map(lambda x: x.split())

In [79]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,prescriptionevents
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,"[MAIN, Tacrolimus, Tacrolimus, DRUG_CD-TACR1, ..."
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,"[MAIN, Warfarin, Warfarin, DRUG_CD-WARF5, GSN-..."
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,"[MAIN, Heparin, Sodium, NaN, DRUG_CD-HEPAPREMI..."
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,"[BASE, D5W, NaN, DRUG_CD-HEPBASE, NaN, NaN, IV..."
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,"[MAIN, Furosemide, Furosemide, DRUG_CD-FURO20,..."


In [80]:
# create output path
mypath_output = "../datasets/prescriptionevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [81]:
# export the dataframe to JSON format
df.to_json(mypath_output+"prescriptionevents.json",orient = 'records') 